In [14]:
#pip install googledrivedownloader
#!pip install PyMuPDF
#!pip install google-auth google-auth-oauthlib google-auth-httplib2
#!pip install google-api-python-client
#!pip install biopython
#!pip install python-docx


In [2]:
from google_drive_downloader import GoogleDriveDownloader as gdd
import os
import PyPDF2
import fitz
from __future__ import print_function
import os.path
import io
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.http import MediaIoBaseDownload
from googleapiclient.http import MediaFileUpload
from Bio import Entrez,  Medline
from docx import Document
import configparser

In [ ]:
os.getcwd()

In [2]:
#set working directory as "final_project"

wd = '/Users/hammedadedamolaakande/Library/CloudStorage/OneDrive-ConcordiaUniversity-Canada/Data_Science_projects/Brave_Career/final_project'

os.chdir(wd)

#### Download ALL the PDFs from the drive to local machine

In [ ]:
#This part is to download ALL the PDFs from the drive to local machine
#Nothing to modify here except the folder ID (if its different from our current project, other remains the same)
#Also, token should be deleted especially if you modify the SCOPES


# If modifying these SCOPES, delete the file token.json.
#SCOPES = ['https://www.googleapis.com/auth/drive']
SCOPES = ['https://www.googleapis.com/auth/drive.readonly']

def create_service():
    creds = None
    # token.json stores my access and refresh tokens, and is
    # created automatically when the authorization flow completes for the first time.
    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file('token.json', SCOPES)
    # If there are no (valid) credentials available, I'm gonna log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)
        # Save the credentials for the next run
        with open('token.json', 'w') as token:
            token.write(creds.to_json())

    service = build('drive', 'v3', credentials=creds)
    return service

# save service instance to interact with Google Drive
service = create_service()



def download_all_pdfs(service, folder_id):
    page_token = None
    while True:
        # Query to list the PDF files in our Google Drive folder
        results = service.files().list(
            q=f"'{folder_id}' in parents and mimeType='application/pdf'",
            pageSize=10,  # Keep as 10 (for fast download)
            fields="nextPageToken, files(id, name)",
            pageToken=page_token  # Use the nextPageToken from the previous request
        ).execute()

        items = results.get('files', [])
        if not items:
            print('No files found.')
            break  # Exit if no files are found

        for item in items:
            file_name = item['name']
            # Check if the file already exists in the current directory
            if not os.path.exists(file_name):
                print(f"Downloading {file_name}...")
                request = service.files().get_media(fileId=item['id'])
                fh = io.BytesIO()
                downloader = MediaIoBaseDownload(fh, request)
                done = False
                while not done:
                    status, done = downloader.next_chunk()
                    print(f"Download {int(status.progress() * 100)}%.")

                # Save the file to disk only if it doesn't already exist
                
                with open(f'PDF_docs/{file_name}', 'wb') as f:
                    fh.seek(0)
                    f.write(fh.read())
            else:
                print(f"{file_name} already exists. Skipping download.")

        page_token = results.get('nextPageToken', None)
        if page_token is None:
            break 

            
download_all_pdfs(service, '1n_9nLO15KnypYNEHbj9Z8USrBoQZVliQ')  

#### Convert the PDFs to text

In [ ]:
#extract data from the current wd- convert the PDFs to text


def is_pdf_file(filename):
    return filename.lower().endswith('.pdf')

def convert_pdf_to_text(pdf_path):
    text = ""
    with fitz.open(pdf_path) as doc:
        for page in doc:
            text += page.get_text()
    return text

def process_pdfs_in_directory(directory_path):
    for filename in os.listdir(directory_path):
        if is_pdf_file(filename):
            full_path = os.path.join(directory_path, filename)
            pdf_text = convert_pdf_to_text(full_path)
            
            # Define a new file name for the output with .txt extension
            output_filename = filename.rsplit('.', 1)[0] + '.txt'
            output_path = os.path.join(directory_path, output_filename)
            
            # Write the extracted text to a new text file
            with open(output_path, 'w', encoding='utf-8') as text_file:
                text_file.write(pdf_text)
            
            print(f"Processed and saved: {filename} as {output_filename}")
            
# Set directory_path to the folder containing the PDFs
directory_path = os.path.join(os.getcwd(), 'PDF_docs')
process_pdfs_in_directory(directory_path)



#### Upload ALL the .txt files from the local machine to drive

In [ ]:
#This part is to upload ALL the .txt files from the local machine to our drive
#Back transfer the text files to the Google Drive folder
#Grant the drive API edit acess and not read only
#Here, I advise you trash the token.json since we're using different SCOPES

SCOPES = ['https://www.googleapis.com/auth/drive']
#SCOPES = ['https://www.googleapis.com/auth/drive.readonly']

def create_service():
    creds = None
    
    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file('token.json', SCOPES)
    # If there are no (valid) credentials available, I'm gonna log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)
        # Save the credentials for the next run
        with open('token.json', 'w') as token:
            token.write(creds.to_json())

    service = build('drive', 'v3', credentials=creds)
    return service

# save service instance
service_upload = create_service()



def upload_all_txt_files(service, folder_id):
    # Get the list of all files in the current working directory
    cwd = os.getcwd()
    files = [f for f in os.listdir(cwd) if os.path.isfile(os.path.join(cwd, f)) and f.endswith('.txt')]

    # List all files in the Google Drive folder to check for duplicates
    existing_files = service.files().list(q=f"'{folder_id}' in parents",
                                          spaces='drive',
                                          fields='nextPageToken, files(id, name)').execute()
    existing_file_names = [file['name'] for file in existing_files.get('files', [])]

    for file_name in files:
        if file_name not in existing_file_names:
            file_path = os.path.join(cwd, file_name)
            file_metadata = {'name': file_name, 'parents': [folder_id]}
            media = MediaFileUpload(file_path, mimetype='text/plain')
            file = service.files().create(body=file_metadata,
                                          media_body=media,
                                          fields='id').execute()
            print(f"{file_name} uploaded successfully with File ID: {file.get('id')}")
        else:
            print(f"{file_name} already exists in the folder. Skipping upload.")


upload_all_txt_files(service_upload, '1n_9nLO15KnypYNEHbj9Z8USrBoQZVliQ')


#### Download ALL the .txt/docx files (not PDFs) from the drive to local machine

In [ ]:
#This part is to download ALL the .txt/docx files (not PDFs) from the drive to local machine
#Again,the folder ID should be the same (our present wd)


# If modifying these SCOPES, delete the file token.json.
SCOPES = ['https://www.googleapis.com/auth/drive.readonly']


def create_service():
    creds = None
    # Recall that token.json stores our access and refresh tokens, and is created automatically when the authorization flow 
    # completes for the first time.
    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file('token.json', SCOPES)
    # If there are no (valid) credentials available, we can log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file('credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)
        # Save the credentials for the next run
        with open('token.json', 'w') as token:
            token.write(creds.to_json())

    service = build('drive', 'v3', credentials=creds)
    return service


# let's save service instance to interact with Google Drive
service = create_service()


def download_all_text_and_docx_files(service, folder_id):
    page_token = None
    while True:
        # Query to list the text and docx files in the specified folder
        results = service.files().list(
            q=f"'{folder_id}' in parents and (mimeType='text/plain' or mimeType='application/vnd.openxmlformats-officedocument.wordprocessingml.document' or mimeType='application/vnd.google-apps.document')",
            pageSize=10,  # Keep as 10 (for fast download)
            fields="nextPageToken, files(id, name, mimeType)",
            pageToken=page_token  # Use the nextPageToken from the previous request
        ).execute()

        items = results.get('files', [])
        if not items:
            print('No files found.')
            break  # Exit if no files are found

        for item in items:
            file_name = item['name']
            file_mime_type = item['mimeType']
            
            # Ensure the file extension is correct for Google Docs files
            if file_mime_type == 'application/vnd.google-apps.document':
                file_name += '.docx'
                request = service.files().export_media(fileId=item['id'], mimeType='application/vnd.openxmlformats-officedocument.wordprocessingml.document')
            else:
                request = service.files().get_media(fileId=item['id'])

            # Check if the file already exists in the current directory
            if not os.path.exists(file_name):
                print(f"Downloading {file_name}...")
                fh = io.BytesIO()
                downloader = MediaIoBaseDownload(fh, request)
                done = False
                while not done:
                    status, done = downloader.next_chunk()
                    print(f"Download {int(status.progress() * 100)}%.")

                # Let's save the file to disk only if it doesn't already exist
                with open(f'txt_docs/{file_name}', 'wb') as f:
                    fh.seek(0)
                    f.write(fh.read())
            else:
                print(f"{file_name} already exists. Skipping download.")

        page_token = results.get('nextPageToken', None)
        if page_token is None:
            break  


# Apply the function
download_all_text_and_docx_files(service, '1n_9nLO15KnypYNEHbj9Z8USrBoQZVliQ')



### Convert docx to txt

In [ ]:
def docx_to_text(docx_file_path):
    doc = Document(docx_file_path)
    text = []
    for paragraph in doc.paragraphs:
        text.append(paragraph.text)
    return '\n'.join(text)

def convert_all_docx_to_txt(folder_path):
    for file_name in os.listdir(folder_path):
        if file_name.endswith('.docx'):
            docx_file_path = os.path.join(folder_path, file_name)
            text_file_name = file_name.replace('.docx', '.txt')
            text_file_path = os.path.join(folder_path, text_file_name)

            # Read the content from the .docx file
            text_content = docx_to_text(docx_file_path)

            # Write the content to a new .txt file
            with open(text_file_path, 'w', encoding='utf-8') as text_file:
                text_file.write(text_content)

            print(f"Converted {file_name} to {text_file_name}")

# Convert all .docx files in the 'txt_docs' folder to .txt files
convert_all_docx_to_txt('txt_docs')


In [ ]:
#Now we can start working on the text data

### Upload Data to AWS

In [5]:
# Data uploader

import boto3

def upload_to_s3(file_name, bucket, object_name=None, aws_access_key_id=None, aws_secret_access_key=None, region_name=None):
    """
    Upload a file to an S3 bucket using put_object

    :param file_name: File to upload
    :param bucket: Bucket to upload to
    :param object_name: S3 object name. If not specified, file_name is used
    :param aws_access_key_id: AWS access key ID
    :param aws_secret_access_key: AWS secret access key
    :param region_name: AWS region name
    :return: True if file was uploaded, else False
    """
    
    # If S3 object_name was not specified, use file_name
    if object_name is None:
        object_name = os.path.basename(file_name)

    # Create an S3 client
    s3_client = boto3.client(
        's3',
        aws_access_key_id=aws_access_key_id,
        aws_secret_access_key=aws_secret_access_key,
        region_name=region_name
    )

    # Upload the file
    with open(file_name, 'rb') as file_data:
        s3_client.put_object(Bucket=bucket, Key=object_name, Body=file_data)
    print(f"File {file_name} uploaded to {bucket}/{object_name}")
    return True



In [17]:
def set_aws_credentials_from_file():
    config = configparser.ConfigParser()

    # Read the AWS credentials and config files
    config.read(os.path.expanduser('~/.aws/credentials'))
    aws_access_key_id = config.get('default', 'aws_access_key_id')
    aws_secret_access_key = config.get('default', 'aws_secret_access_key')

    config.read(os.path.expanduser('~/.aws/config'))
    region_name = config.get('default', 'region')

    # Set environment variables
    os.environ['AWS_ACCESS_KEY_ID'] = aws_access_key_id
    os.environ['AWS_SECRET_ACCESS_KEY'] = aws_secret_access_key
    os.environ['AWS_REGION'] = region_name

def main():
    set_aws_credentials_from_file()

    folder_path = "./txt_docs/"  # Local folder path
    bucket_name = "chatbot-pro"
    aws_access_key_id = os.getenv('AWS_ACCESS_KEY_ID')
    aws_secret_access_key = os.getenv('AWS_SECRET_ACCESS_KEY')
    region_name = os.getenv('AWS_REGION')
    s3_folder = "raw-data/"  # S3 folder path

    # Check if AWS credentials are available
    if not aws_access_key_id or not aws_secret_access_key or not region_name:
        print("AWS credentials are not set in the environment variables.")
        return

    # Upload all .txt files to S3
    for file_name in os.listdir(folder_path):
        if file_name.endswith('.txt'):
            full_file_path = os.path.join(folder_path, file_name)
            object_name = os.path.join(s3_folder, file_name)  # S3 object name with folder path
            upload_to_s3(
                file_name=full_file_path,
                bucket=bucket_name,
                object_name=object_name,
                aws_access_key_id=aws_access_key_id,
                aws_secret_access_key=aws_secret_access_key,
                region_name=region_name
            )

if __name__ == '__main__':
    main()

File ./txt_docs/A1C Diabetes Test.txt uploaded to chatbot-pro/raw-data/A1C Diabetes Test.txt
File ./txt_docs/Diabetes_symptoms_and_causes.txt uploaded to chatbot-pro/raw-data/Diabetes_symptoms_and_causes.txt
File ./txt_docs/A1C_test_variants_and_ethnicity.txt uploaded to chatbot-pro/raw-data/A1C_test_variants_and_ethnicity.txt
File ./txt_docs/Diabetes_brochure 5 15 2020.txt uploaded to chatbot-pro/raw-data/Diabetes_brochure 5 15 2020.txt
File ./txt_docs/Managing_Diabetes.txt uploaded to chatbot-pro/raw-data/Managing_Diabetes.txt
File ./txt_docs/Type_1_Diabetes.txt uploaded to chatbot-pro/raw-data/Type_1_Diabetes.txt
File ./txt_docs/Diabetes_and_Digestion.txt uploaded to chatbot-pro/raw-data/Diabetes_and_Digestion.txt
File ./txt_docs/Diabetes_Healthy_Living.txt uploaded to chatbot-pro/raw-data/Diabetes_Healthy_Living.txt
File ./txt_docs/Diabetes_Management_Travelling.txt uploaded to chatbot-pro/raw-data/Diabetes_Management_Travelling.txt
File ./txt_docs/Medicines.txt uploaded to chatbot